In [23]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
%pip install shap
import shap
import warnings

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\victo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
df = pd.read_csv('dataset.csv')

In [25]:
print(df.columns.tolist())

['Marital status', 'Application mode', 'Application order', 'Course', 'Daytime/evening attendance', 'Previous qualification', 'Nacionality', "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation", 'Displaced', 'Educational special needs', 'Debtor', 'Tuition fees up to date', 'Gender', 'Scholarship holder', 'Age at enrollment', 'International', 'Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)', 'Unemployment rate', 'Inflation rate', 'GDP', 'Target']


In [26]:
df['evasao'] = df['Target'].apply(
    lambda x: 1 if x == 'Dropout' else 0 if x == 'Graduate' else np.nan
).astype('Int64')
df_clean = df.dropna(subset=['evasao']).copy()


In [41]:
print("\nDistribuição da variável target:")
print(df_clean['evasao'].value_counts().rename(index={0: 'Não Evasão', 1: 'Evasão'}))
print(f"Taxa de evasão: {df_clean['evasao'].mean():.2%}")


Distribuição da variável target:
evasao
Não Evasão    2209
Evasão        1421
Name: count, dtype: Int64
Taxa de evasão: 39.15%


In [28]:
features_keep = [
    'Age at enrollment',
    'Gender',
    'Daytime/evening attendance',
    'Scholarship holder',
    'Educational special needs',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (grade)'
]

X = df_clean[features_keep].copy()
y = df_clean['evasao']

In [29]:
numeric_features = [
    'Age at enrollment',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (grade)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (grade)'
]

categorical_features = [
    'Gender',
    'Daytime/evening attendance',
    'Scholarship holder',
    'Educational special needs'
]

In [30]:
print(f"Variáveis numéricas: {len(numeric_features)}")
print(f"Variáveis categóricas: {len(categorical_features)}")

Variáveis numéricas: 7
Variáveis categóricas: 4


In [31]:
missing_values = X.isnull().sum()
total_missing = missing_values.sum()

if total_missing > 0:
    print(f"\n⚠️ Valores ausentes detectados: {total_missing}")
    print(missing_values[missing_values > 0])
else:
    print("\n✅ Nenhum valor ausente encontrado.")



✅ Nenhum valor ausente encontrado.


In [32]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())  
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDivisão dos dados:")
print(f"Treino: {X_train.shape[0]} amostras ({y_train.mean():.2%} evasão)")
print(f"Teste:  {X_test.shape[0]} amostras ({y_test.mean():.2%} evasão)")


Divisão dos dados:
Treino: 2904 amostras (39.15% evasão)
Teste:  726 amostras (39.12% evasão)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

print("\nOTIMIZANDO GRADIENT BOOSTING...")
print("="*50)

gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

param_grid = {
    'classifier__n_estimators': [200, 300],
    'classifier__learning_rate': [0.05, 0.1],
    'classifier__max_depth': [3, 4],
    'classifier__min_samples_leaf': [20, 50],
    'classifier__subsample': [0.8, 1.0],
}

print("Executando Grid Search...")
grid_search = GridSearchCV(
    estimator=gb_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

best_gb_model = grid_search.best_estimator_
print(f"\nMelhores parâmetros:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")



🚀 OTIMIZANDO GRADIENT BOOSTING...
⏳ Executando Grid Search...
Fitting 5 folds for each of 32 candidates, totalling 160 fits

🎯 Melhores parâmetros:
  classifier__learning_rate: 0.05
  classifier__max_depth: 3
  classifier__min_samples_leaf: 50
  classifier__n_estimators: 200
  classifier__subsample: 1.0


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    average_precision_score
)

print("\nAVALIAÇÃO FINAL DO MODELO OTIMIZADO:")
print("="*50)

y_pred = best_gb_model.predict(X_test)
y_proba = best_gb_model.predict_proba(X_test)[:, 1]

metrics = {
    'AUC': roc_auc_score(y_test, y_proba),
    'PR-AUC': average_precision_score(y_test, y_proba),
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, zero_division=0),
    'Recall': recall_score(y_test, y_pred, zero_division=0),
    'F1-Score': f1_score(y_test, y_pred, zero_division=0)
}

print("\nMétricas Finais:")
for metric, value in metrics.items():
    print(f"{metric:12}: {value:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Não Evasão', 'Evasão']))

print("\n🔢 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

tn, fp, fn, tp = cm.ravel()
print(f"\nInterpretação:")
print(f"  Verdadeiros Negativos (TN): {tn}")
print(f"  Falsos Positivos (FP): {fp}")
print(f"  Falsos Negativos (FN): {fn}")
print(f"  Verdadeiros Positivos (TP): {tp}")



📊 AVALIAÇÃO FINAL DO MODELO OTIMIZADO:

📈 Métricas Finais:
AUC         : 0.9615
PR-AUC      : 0.9583
Accuracy    : 0.9008
Precision   : 0.8841
Recall      : 0.8592
F1-Score    : 0.8714

📋 Classification Report:
              precision    recall  f1-score   support

  Não Evasão       0.91      0.93      0.92       442
      Evasão       0.88      0.86      0.87       284

    accuracy                           0.90       726
   macro avg       0.90      0.89      0.90       726
weighted avg       0.90      0.90      0.90       726


🔢 Confusion Matrix:
[[410  32]
 [ 40 244]]

Interpretação:
  Verdadeiros Negativos (TN): 410
  Falsos Positivos (FP): 32
  Falsos Negativos (FN): 40
  Verdadeiros Positivos (TP): 244


In [ ]:
print("\nIMPORTÂNCIA DAS FEATURES:")
print("="*50)

preprocessor = best_gb_model.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()

importances = best_gb_model.named_steps['classifier'].feature_importances_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\nTop 10 Features mais importantes:")
print(feature_importance_df.head(10))



🔍 IMPORTÂNCIA DAS FEATURES:

Top 10 Features mais importantes:
                                    feature  importance
4  num__Curricular units 2nd sem (approved)    0.778213
2  num__Curricular units 1st sem (enrolled)    0.049346
1  num__Curricular units 1st sem (approved)    0.046671
5  num__Curricular units 2nd sem (enrolled)    0.037272
6     num__Curricular units 2nd sem (grade)    0.030716
0                    num__Age at enrollment    0.021103
9                 cat__Scholarship holder_1    0.018134
3     num__Curricular units 1st sem (grade)    0.014188
7                             cat__Gender_1    0.003018
8         cat__Daytime/evening attendance_1    0.001339


In [45]:
print("\n💾 Salvando modelo...")

import joblib
import json

model_filename = 'gradient_boosting_dropout.pkl'
joblib.dump(best_gb_model, model_filename)
print(f"✅ Modelo salvo como: {model_filename}")

# ===== FUNÇÃO DE PREDIÇÃO =====
def predict_dropout(model, new_data, features_expected=None):
    """
    Faz predições de evasão para novos dados.
    
    Args:
        model: Modelo treinado (pipeline completo)
        new_data: dict ou DataFrame com novos dados
        features_expected: lista de colunas esperadas pelo modelo (opcional)
    
    Returns:
        tuple: (predições, probabilidades)
    """
    import pandas as pd

    if not isinstance(new_data, pd.DataFrame):
        new_data = pd.DataFrame([new_data])

    if features_expected:
        for col in features_expected:
            if col not in new_data.columns:
                new_data[col] = None  
        new_data = new_data[features_expected]

    probabilities = model.predict_proba(new_data)[:, 1]
    predictions = model.predict(new_data)

    return predictions, probabilities

print(f"\n🎉 MODELO FINALIZADO!")
print(f"✅ AUC Score: {metrics['AUC']:.4f}")
print(f"✅ F1 Score: {metrics['F1-Score']:.4f}")
print(f"✅ Precision: {metrics['Precision']:.4f}")
print(f"✅ Recall: {metrics['Recall']:.4f}")

print(f"\n💡 Para usar o modelo em novos dados:")
print(f"loaded_model = joblib.load('{model_filename}')")



💾 Salvando modelo...
✅ Modelo salvo como: gradient_boosting_dropout.pkl

🎉 MODELO FINALIZADO!
✅ AUC Score: 0.9615
✅ F1 Score: 0.8714
✅ Precision: 0.8841
✅ Recall: 0.8592

💡 Para usar o modelo em novos dados:
loaded_model = joblib.load('gradient_boosting_dropout.pkl')


In [46]:

FEATURES_EXPECTED = X.columns.tolist()

def predict_student_dropout(student_dict, model=None, threshold_low=0.4, threshold_high=0.7):
    """
    Recebe um dicionário com os dados do estudante (somente features lite)
    e retorna a predição de evasão + probabilidade + faixa de risco.
    """
    import numpy as np
    import pandas as pd

    if model is None:
        model = best_gb_model  

    df_in = pd.DataFrame([student_dict])
    for col in FEATURES_EXPECTED:
        if col not in df_in.columns:
            df_in[col] = np.nan
    df_in = df_in[FEATURES_EXPECTED]

    proba = float(model.predict_proba(df_in)[:, 1][0])
    pred = int(model.predict(df_in)[0])

    label = "Evasão" if pred == 1 else "Não Evasão"
    if proba >= threshold_high:
        risk = "Alto"
    elif proba >= threshold_low:
        risk = "Médio"
    else:
        risk = "Baixo"

    return {"prediction": label, "probability": proba, "confidence": f"{proba:.2%}", "risk_level": risk}


def create_fictional_student_lite():
    """
    Cria um estudante fictício com os CAMPOS usados no modelo lite (início do 3º período).
    """
    return {
        
        'Age at enrollment': 19,
        'Gender': 1,  
        'Daytime/evening attendance': 1, 
        'Scholarship holder': 0,
        'Educational special needs': 0,

        'Curricular units 1st sem (approved)': 5,
        'Curricular units 1st sem (enrolled)': 6,
        'Curricular units 1st sem (grade)': 12.5,

        'Curricular units 2nd sem (approved)': 4,
        'Curricular units 2nd sem (enrolled)': 6,
        'Curricular units 2nd sem (grade)': 11.2,
    }


print("🧑‍🎓 CRIANDO E TESTANDO ESTUDANTE FICTÍCIO (versão lite)")
print("="*50)

estudante_teste = create_fictional_student_lite()

aprov1 = estudante_teste['Curricular units 1st sem (approved)']
mat1  = estudante_teste['Curricular units 1st sem (enrolled)']
nota1 = estudante_teste['Curricular units 1st sem (grade)']
aprov2 = estudante_teste['Curricular units 2nd sem (approved)']
mat2  = estudante_teste['Curricular units 2nd sem (enrolled)']
nota2 = estudante_teste['Curricular units 2nd sem (grade)']

rate1 = aprov1 / mat1 if mat1 else 0.0
rate2 = aprov2 / mat2 if mat2 else 0.0

print("📋 Perfil do Estudante:")
print(f"  👤 Idade na matrícula: {estudante_teste['Age at enrollment']}")
print(f"  🕒 Turno: {'Diurno' if estudante_teste['Daytime/evening attendance']==1 else 'Noturno'}")
print(f"  🎓 Bolsista: {'Sim' if estudante_teste['Scholarship holder'] else 'Não'}")
print(f"  📊 1º sem: {aprov1}/{mat1} aprovadas ({rate1:.0%}) | média {nota1}")
print(f"  📊 2º sem: {aprov2}/{mat2} aprovadas ({rate2:.0%}) | média {nota2} (Δ nota {nota2-nota1:+.1f})")

print("\n🔮 PREDIÇÃO DO MODELO:")
print("="*30)
resultado = predict_student_dropout(estudante_teste)

print(f"🎯 Predição: {resultado['prediction']}")
print(f"📈 Probabilidade: {resultado['confidence']}")
print(f"⚠️ Nível de Risco: {resultado['risk_level']}")

print("\n🔍 ANÁLISE DETALHADA:")
risk_factors = []
protective_factors = []

if rate1 < 0.8:
    risk_factors.append("❌ Taxa de aprovação baixa no 1º semestre")
if rate2 < rate1:
    risk_factors.append("❌ Queda na taxa de aprovação no 2º semestre")
if nota2 < nota1:
    risk_factors.append("❌ Queda de notas do 1º para o 2º semestre")
if estudante_teste['Scholarship holder'] == 1:
    protective_factors.append("✅ Bolsa (apoio pode reduzir risco)")
if estudante_teste['Educational special needs'] == 0:
    protective_factors.append("✅ Sem necessidades educacionais especiais")
if estudante_teste['Age at enrollment'] <= 20:
    protective_factors.append("✅ Idade típica de ingresso")

print("⚠️ Fatores de Risco:")
print("   - " + "\n   - ".join(risk_factors) if risk_factors else "   (nenhum crítico)")

print("✅ Fatores Protetivos:")
print("   - " + "\n   - ".join(protective_factors) if protective_factors else "   (nenhum listado)")

print("\n🔄 TESTANDO OUTROS CENÁRIOS:")
print("="*30)

estudante_alto = estudante_teste.copy()
estudante_alto['Curricular units 1st sem (approved)'] = 2
estudante_alto['Curricular units 2nd sem (approved)'] = 1
estudante_alto['Curricular units 2nd sem (grade)'] = 8.5
res_alto = predict_student_dropout(estudante_alto)
print(f"🔴 Alto risco → {res_alto['prediction']} ({res_alto['confidence']})")

estudante_baixo = estudante_teste.copy()
estudante_baixo['Curricular units 1st sem (approved)'] = estudante_baixo['Curricular units 1st sem (enrolled)']
estudante_baixo['Curricular units 2nd sem (approved)'] = estudante_baixo['Curricular units 2nd sem (enrolled)']
estudante_baixo['Curricular units 2nd sem (grade)'] = max(15.0, nota2 + 2.0)
res_baixo = predict_student_dropout(estudante_baixo)
print(f"🟢 Baixo risco → {res_baixo['prediction']} ({res_baixo['confidence']})")


🧑‍🎓 CRIANDO E TESTANDO ESTUDANTE FICTÍCIO (versão lite)
📋 Perfil do Estudante:
  👤 Idade na matrícula: 19
  🕒 Turno: Diurno
  🎓 Bolsista: Não
  📊 1º sem: 5/6 aprovadas (83%) | média 12.5
  📊 2º sem: 4/6 aprovadas (67%) | média 11.2 (Δ nota -1.3)

🔮 PREDIÇÃO DO MODELO:
🎯 Predição: Evasão
📈 Probabilidade: 55.40%
⚠️ Nível de Risco: Médio

🔍 ANÁLISE DETALHADA:
⚠️ Fatores de Risco:
   - ❌ Queda na taxa de aprovação no 2º semestre
   - ❌ Queda de notas do 1º para o 2º semestre
✅ Fatores Protetivos:
   - ✅ Sem necessidades educacionais especiais
   - ✅ Idade típica de ingresso

🔄 TESTANDO OUTROS CENÁRIOS:
🔴 Alto risco → Evasão (99.25%)
🟢 Baixo risco → Não Evasão (7.06%)


In [47]:
import json, joblib
from pathlib import Path
from sklearn.metrics import precision_recall_curve
import numpy as np
import pandas as pd

print("\n💾 Salvando modelo + metadados...")

MODEL_PATH = Path("gradient_boosting_dropout.pkl")
META_PATH  = Path("model_meta.json")

joblib.dump(best_gb_model, MODEL_PATH)
print(f"✅ Modelo salvo em: {MODEL_PATH}")

try:
    _prec, _rec, _thr = precision_recall_curve(y_test, y_proba)
    _f1s = 2 * _prec * _rec / (np.clip(_prec + _rec, 1e-9, None))
    _best_idx = int(np.argmax(_f1s))
    best_threshold = float(_thr[_best_idx]) if _best_idx < len(_thr) else 0.5
except Exception:
    best_threshold = 0.5  

risk_bands = {
    "low": 0.40,
    "high": 0.70
}

meta = {
    "model_file": str(MODEL_PATH),
    "features_expected": list(X.columns),
    "metrics": metrics,             
    "best_threshold": best_threshold,
    "risk_bands": risk_bands
}
with open(META_PATH, "w") as f:
    json.dump(meta, f, indent=2)
print(f"✅ Metadados salvos em: {META_PATH}")


def predict_with_bands(new_data, model_path=str(MODEL_PATH), meta_path=str(META_PATH)):
    """
    Carrega modelo e metadados, aceita dict ou DataFrame, ajusta colunas,
    retorna rótulo, probabilidade e banda de risco.
    """
    mdl = joblib.load(model_path)
    with open(meta_path, "r") as f:
        m = json.load(f)

    features_expected = m["features_expected"]
    best_thr = float(m.get("best_threshold", 0.5))
    bands = m.get("risk_bands", {"low": 0.4, "high": 0.7})

    if not isinstance(new_data, pd.DataFrame):
        new_data = pd.DataFrame([new_data])

    for col in features_expected:
        if col not in new_data.columns:
            new_data[col] = np.nan
    X_in = new_data[features_expected]

    proba = float(mdl.predict_proba(X_in)[:, 1][0])
    pred  = int(proba >= best_thr)

    label = "Evasão" if pred == 1 else "Não Evasão"
    if proba >= bands["high"]:
        band = "Alto"
    elif proba >= bands["low"]:
        band = "Médio"
    else:
        band = "Baixo"

    return {
        "prediction": label,
        "probability": proba,
        "confidence": f"{proba:.2%}",
        "risk_level": band,
        "threshold_used": best_thr,
        "bands": bands
    }

print("\n🧪 Exemplo rápido com predict_with_bands:")
res = predict_with_bands(estudante_teste)
print(res)



💾 Salvando modelo + metadados...
✅ Modelo salvo em: gradient_boosting_dropout.pkl
✅ Metadados salvos em: model_meta.json

🧪 Exemplo rápido com predict_with_bands:
{'prediction': 'Não Evasão', 'probability': 0.5539532236946263, 'confidence': '55.40%', 'risk_level': 'Médio', 'threshold_used': 0.6102142877598777, 'bands': {'low': 0.4, 'high': 0.7}}
